In [1]:
import os
import pandas as pd

path = os.path.abspath("/usr/xtmp/aak61/CUB_200_2011/datasets/cub200_cw/")
train_path = os.path.join(path, "train.parquet")
test_path = os.path.join(path, "test.parquet")

train = pd.read_parquet(train_path)
print(train.columns)
train.head()

Index(['image_id', 'certainty_id', 'class', 'path', 'is_train', 'low_level',
       'high_level', 'coords', 'bbox'],
      dtype='object')


,image_id,certainty_id,class,path,is_train,low_level,high_level,coords,bbox
9,2,4,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,has_wing_color::grey,wing,"[168.0, 62.0, 248.0, 142.0]","[139.0, 30.0, 153.0, 264.0]"
10,2,4,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,has_wing_color::grey,wing,"[194.0, 153.0, 274.0, 233.0]","[139.0, 30.0, 153.0, 264.0]"
11,2,4,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,has_back_color::grey,back,"[188.0, 98.0, 268.0, 178.0]","[139.0, 30.0, 153.0, 264.0]"
12,2,4,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,has_tail_shape::notched_tail,tail,"[123.0, 115.0, 203.0, 195.0]","[139.0, 30.0, 153.0, 264.0]"
13,2,4,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,has_upper_tail_color::grey,tail,"[123.0, 115.0, 203.0, 195.0]","[139.0, 30.0, 153.0, 264.0]"


In [2]:
train[train["coords"].isna()]

,image_id,certainty_id,class,path,is_train,low_level,high_level,coords,bbox


In [ ]:
# Confirming no overlap
train_ids = pd.Series(train["image_id"].unique())
test_ids = pd.Series(test["image_id"].unique())
print(train_ids.isin(test_ids).sum())
print(len(train_ids))
print(len(test_ids))

In [4]:
from PIL import Image, ImageDraw
import os

base_path = "/usr/xtmp/aak61/CUB_200_2011/images/"

# Assuming d is a dataframe and you have already imported necessary libraries to handle it
selected_data = test_df[test_df['image_id'] == 1].iloc[0]
#selected_data = d[d['image_id'] == 12].iloc[0]

# Load image
#img_path = os.path.join(base_path, selected_data["path"])
img_path = selected_data["path"]
print(img_path)
image = Image.open(img_path)
draw = ImageDraw.Draw(image)

# Print size of the image
print("Image size:", image.size)

# Extract box coordinates and draw cyan dots at the top right and bottom right
box = selected_data['bbox']  # Assuming this is another set of coordinates

# Calculate the center of the coordinates
coords = selected_data['coords']
x_scale = 224/box[2]
y_scale = 224/box[3]

print(x_scale)
print(y_scale)
coords1 = [
    coords[0] * x_scale,
    coords[1] * y_scale,
    coords[2] * x_scale,
    coords[3] * y_scale
]

# Draw a red dot at the center
draw.ellipse([(coords[0] - 5, coords[1] - 5), (coords[0] + 5, coords[1] + 5)], fill='red')
draw.ellipse([(coords[2] - 5, coords[3] - 5), (coords[2] + 5, coords[3] + 5)], fill='red')

print(selected_data["high_level"])

# Display the image
image_path = "output_image.png"  # You can change the path and file name as needed
image.save(image_path)
print(f"Image saved to {image_path}")


NameError: name 'test_df' is not defined

In [3]:
import pandas as pd
import os
from PIL import Image, ImageDraw, ImageFilter
import numpy as np
from tqdm import tqdm

def preprocess_and_augment(df, base_path, target_size=(224, 224)):
    augmented_rows = []
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):

        original_path = os.path.join(base_path, f"images/{row['path']}")
        new_dir = os.path.join(base_path, 'datasets', 'cub200_cw', 'train' if row['is_train'] else 'test')
        new_path = os.path.join(new_dir, f"{row['image_id']}.jpg")
        
        if not os.path.exists(new_dir):
            os.makedirs(new_dir)
        
        bbox = row['bbox']
        coords = row['coords']

        if not os.path.exists(new_path):
            image = Image.open(original_path)
            image_cropped = image.crop((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]))
            image_resized = image_cropped.resize(target_size)
            image_resized.save(new_path)
        
        x_scale = target_size[0] / bbox[2]
        y_scale = target_size[1] / bbox[3]
        
        new_coords = [
            int((coords[0] - bbox[0]) * x_scale),
            int((coords[1] - bbox[1]) * y_scale),
            int((coords[2] - bbox[0]) * x_scale),
            int((coords[3] - bbox[1]) * y_scale)
        ]

        # Update df
        df.at[idx, 'coords'] = new_coords
        df.at[idx, 'path'] = new_path
        
        #if row['is_train']:
        #    augmented_rows.extend(augment_data(image_resized, row, new_dir, new_coords))

    # Add augmented rows to the original DataFrame
    #augmented_df = pd.DataFrame(augmented_rows)
    #return pd.concat([df, augmented_df], ignore_index=True)
    return df

def augment_data(image, original_row, dir_path, coords):
    transformations = {
        'flipped': Image.FLIP_LEFT_RIGHT,
        'rotated': 15,  # degrees
        'noisy': 'add_noise',
        'blurred': (ImageFilter.GaussianBlur(radius=1))
    }
    
    new_rows = []
    for suffix, transform in transformations.items():
        if suffix == 'rotated':
            new_image = image.rotate(transform, expand=True)
            new_coords = [coords[0], coords[1], coords[2], coords[3]]  # Placeholder: update for correct rotation
        elif suffix == 'noisy':
            new_image = add_noise(image)
            new_coords = coords
        elif suffix == 'flipped':
            new_image = image.transpose(transform)
            new_coords = [image.width - coords[2], coords[1], image.width - coords[0], coords[3]]
        else:
            new_image = image.filter(transform)
            new_coords = coords
        
        new_path = os.path.join(dir_path, f"{original_row['image_id']}_{suffix}.jpg")
        new_image.save(new_path)
        
        # Clone original row and update necessary fields
        new_row = original_row.copy()
        new_row['image_id'] = f"{original_row['image_id']}_{suffix}"
        new_row['path'] = new_path
        new_row['coords'] = new_coords
        new_rows.append(new_row)
    
    return new_rows

def add_noise(image):
    np_image = np.array(image)
    noise = np.random.normal(loc=0, scale=25, size=np_image.shape)
    np_image = np.clip(np_image + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(np_image)

# Example usage
base_path = os.path.abspath("/usr/xtmp/aak61/CUB_200_2011/")
data_path = os.path.join(base_path, "datasets/cub200_cw")
train_path = os.path.join(path, "train.parquet")
test_path = os.path.join(path, "test.parquet")
train_df = pd.read_parquet(train_path)
test_df = pd.read_parquet(test_path)

# Concatenate train and test dataframes
#full_df = pd.concat([train_df, test_df])
#print(full_df.shape)
t = preprocess_and_augment(train_df, base_path)


100%|██████████| 149996/149996 [01:33<00:00, 1601.63it/s]


In [4]:
t.to_parquet(train_path)